# Importamos librerias

Las librerias en python son lo que traen a nuestro ambiente funciones ya programadas listas para usarlas.

In [1]:
import pandas as pd
import re

# Cargar datos



1.   Primero vamos a **importar nuestros datos** al espacio de trabajo: desplazamos el archivo hacia el cuadro de la izquierda.
2.   Vamos a usar la libreria pandas de python: con **pd.read_csv** levantamos el csv deseado.
3. Seleccionamos nuestros campos de interes



In [2]:
data=pd.read_csv("data.csv",encoding="UTF-8")

In [3]:
tamaño=data.shape
print(f"Tamaño data {tamaño}")

Tamaño data (13, 2)


In [4]:
columnas=data.columns
columnas

Index(['ID', 'Domicilio'], dtype='object')

#### Campos de interes

In [5]:
data["Domicilio"]

0     Luis Viale 2465 Planta baja departamento 8
1               Luis Viale         2465 Pb Dto 3
2                                Galiciiiiia 546
3                      Aguero                 77
4                           Luis viale 1900 2 26
5           Avenida san martin 2172 1 Piso Dto 7
6                      Aristobulo del valle 2458
7                                    VERAA  3016
8                   Almt. J. Francisco Segui 232
9                             Alvatez Jonte 3594
10                             Pacheco 286 9No A
11         Gral paz 394, C1202 Abb, Buenos Aires
12                                          2300
Name: Domicilio, dtype: object

# Limpieza estandar de direcciones

Acá se pueden elegir las tareas de limpieza que se realizarán usaremos la libreria **re** de **regex** en donde usaremos expresiones regulares para indicar que queremos limpiar.

* [Acá](https://support.google.com/a/answer/1371417?hl=es) hay un lindo tutorial de Google.

* [Acá](https://www3.ntu.edu.sg/home/ehchua/programming/howto/Regexe.html) hay un buen machete para consultar.

* Googlear que queremos hacer junto al indicador **regex**

#### Ejemplo regex

In [6]:
x=re.sub("hola", "chau", "hola, vivo en capital federal")
x

'chau, vivo en capital federal'

#### Ejemplo regex 2 

In [7]:
x=re.sub("^(\\D*\\d+).*", "\\1", "Luis Viale 246 Planta baja departamento 8")
#x=re.sub("baja","",x)
#x=re.sub("2465","",x)
x

'Luis Viale 246'

#### Ahora si, limpiamos

In [8]:
def calles_limpias(a):
    replacements = [        
        ("[^\w\s]", ""),#  Saco todo lo que no sean letras o numeros (puntuación).
        (r"\s+", " "),#  Si hay muchos espacios, me quedo con uno solo.
        (r"^[0-9]+", ''),#  Saco numeros del inicio (^) de la direccion.
        (r'([A-Z])\1+', r'\1'), #elimino letras duplicadas
        ("^(\\D*\\d+).*", "\\1"),#  Me quedo con el primer grupo que sea un conjunto de letras seguidos por numeros
    ]

    def clean(e):
        e = e.upper()
        for ptr, rpl in replacements:
            e = e.strip()#  Saco espacios del inicio y fin de la direccion
            e = re.sub(ptr, rpl, e)
        return e

    return [clean(element.upper()) for element in a]#  Convierto todo a mayuscula antes de limpiar

### Probamos a ver como anda

In [9]:
calles_limpias(["Luis Viale 246 Planta baja Dto 3"])

['LUIS VIALE 246']

### Limpiamos los datos

In [10]:
data.Domicilio=data.Domicilio.astype({'Domicilio': str}).reset_index(drop=True)
data["direccion_limpia"]=calles_limpias(data.Domicilio) # Aplico limpieza
data

,ID,Domicilio,direccion_limpia
0,1,Luis Viale 2465 Planta baja departamento 8,LUIS VIALE 2465
1,2,Luis Viale 2465 Pb Dto 3,LUIS VIALE 2465
2,3,Galiciiiiia 546,GALICIA 546
3,4,Aguero 77,AGUERO 77
4,5,Luis viale 1900 2 26,LUIS VIALE 1900
5,6,Avenida san martin 2172 1 Piso Dto 7,AVENIDA SAN MARTIN 2172
6,7,Aristobulo del valle 2458,ARISTOBULO DEL VALE 2458
7,8,VERAA 3016,VERA 3016
8,9,Almt. J. Francisco Segui 232,ALMT J FRANCISCO SEGUI 232
9,10,Alvatez Jonte 3594,ALVATEZ JONTE 3594


In [11]:
# Remuevo filas en las que quedaron solo numeros
data_limpia=data[[False if str(x).isdigit() else True for x in data.direccion_limpia]].reset_index(drop=True)
# Datos que no pudieron limpiarse
data_limpia=data_limpia[data_limpia.direccion_limpia!=""].reset_index(drop=True)

In [12]:
# Proporcion de datos que pudieron limpiarse
len(data_limpia.direccion_limpia)/len(data.direccion_limpia)

0.9230769230769231

## Extra: remover acentos y cambiar ñ por n

In [13]:
#data_limpia.direccion_limpia=[x.upper().translate(str.maketrans('ÁÉÍÓÚÑ','AEIOUN')).strip() for x in data_limpia.direccion_limpia]

## Guardamos data sucia

In [14]:
data_sucia=pd.concat([data[[True if str(x).isdigit() else False for x in data.direccion_limpia]],
                      data[data.direccion_limpia==""]])

In [15]:
data_sucia.to_csv("data_sucia.csv")

## Extra: Limpiamos alturas que no sean numeros

In [16]:
#Removemos alturas erroneas
#data=data[data['altura'].apply(lambda x: str(x).isdigit())].astype({'altura': 'int32'}).reset_index(drop=True)

# Guardar data limpia

In [17]:
data_limpia.to_csv("data_limpia.csv")